In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 225kB 8.7MB/s 
     |████████████████████████████████| 1.8MB 34.6MB/s 
     |████████████████████████████████| 122kB 48.7MB/s 
     |████████████████████████████████| 7.8MB 50.0MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 3.3MB 44.5MB/s 
     |████████████████████████████████| 81kB 11.8MB/s 
     |████████████████████████████████| 245kB 54.2MB/s 
     |████████████████████████████████| 1.2MB 45.0MB/s 
     |████████████████████████████████| 2.5MB 45.1MB/s 
     |████████████████████████████████| 133kB 48.4MB/s 
     |████████████████████████████████| 102kB 12.9MB/s 
     |████████████████████████████████| 163kB 50.0MB/s 
     |████████████████████████████████| 4.2MB 42.2MB/s 
     |████████████████████████████████| 81kB 11.4MB/s 
     |████████████████████████████████| 112kB 57.3MB/s 
     |████████████████████████████████| 245kB 50.3MB/s 
     |████████████████████████████████| 122kB 52.0MB/

In [4]:
import pandas as pd

from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split

In [ ]:
# Import data
df = pd.read_csv('/content/drive/MyDrive/app_reviews_all_annotated2.csv')
df = df[['review', 'argument_cat', 'decision_cat']]

# Remove missing rows
df = df.dropna()

df = df.groupby('argument_cat').filter(lambda x : len(x) > 1)
df = df.groupby('decision_cat').filter(lambda x : len(x) > 1)

# Convert to numeric for bert
df['Argument'] = pd.Categorical(df['argument_cat'])
df['Decision'] = pd.Categorical(df['decision_cat'])
df['argument_cat'] = df['Argument'].cat.codes
df['decision_cat'] = df['Decision'].cat.codes
#df = df[['review', 'argument_cat', 'decision_cat']]
df = df[['review', 'argument_cat']]
df

,review,argument_cat
5,Everyone should use it Full Review,2
37,Improve face detection please !!! Full Rev...,2
192,I think we just need more updates Full Rev...,2
201,if you're looking for an alternative for Pho...,1
221,it's the best photo editor app i've ever see...,1
...,...,...
45986,OMG IT'S SO AWESOME YOU SHOULD TRY IT Full...,1
46051,I litterly love this game i use to play this...,1
46076,"Please make more levels. Other wise, it's go...",1
46077,awesome game worth the money! Full Review,1


In [ ]:
train_df, eval_df = train_test_split(df, test_size = 0.1, stratify=df[['argument_cat']])

In [ ]:


# Train and Evaluation data needs to be in a Pandas Dataframe containing at least two columns. If the Dataframe has a header, it should contain a 'text' and a 'labels' column. If no header is present, the Dataframe should contain at least two columns, with the first column is the text with type str, and the second column in the label with type int.
#train_data = [
#    ["Example sentence belonging to class 1", 1],
#    ["Example sentence belonging to class 0", 0],
#    ["Example eval senntence belonging to class 2", 2],
#]
#train_df = pd.DataFrame(train_data)
#
#eval_data = [
#    ["Example eval sentence belonging to class 1", 1],
#    ["Example eval sentence belonging to class 0", 0],
#    ["Example eval senntence belonging to class 2", 2],
#]
#eval_df = pd.DataFrame(eval_data)

# Create a ClassificationModel
model = ClassificationModel(
    "distilbert",
    "distilbert-base-uncased",
    num_labels=4,
    args={"reprocess_input_data": True, "overwrite_output_dir": True},
)

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

#predictions, raw_outputs = model.predict(["Some arbitary sentence"])

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

  0%|          | 0/2488 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/311 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1384: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/277 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/35 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='weighted')

def f1_multiclass2(labels, preds):
    return f1_score(labels, preds, average='micro')

def f1_multiclass3(labels, preds):
    return f1_score(labels, preds, average='macro')
    
result, model_outputs, wrong_predictions = model.eval_model(eval_df, f1=f1_multiclass, acc=accuracy_score)
result2, model_outputs, wrong_predictions = model.eval_model(eval_df, f1=f1_multiclass2, acc=accuracy_score)
result3, model_outputs, wrong_predictions = model.eval_model(eval_df, f1=f1_multiclass3, acc=accuracy_score)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1384: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/277 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/277 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/277 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/35 [00:00<?, ?it/s]

In [ ]:
result

{'acc': 0.9061371841155235,
 'eval_loss': 0.2610128658424531,
 'f1': 0.9034748904825386,
 'mcc': 0.6329296192212891}

In [ ]:
print('------Argument Classification-------\n')
print('weighted F1-score: ' + str(result['f1']))
print('micro F1-score: ' + str(result2['f1']))
print('macro F1-score: ' + str(result3['f1']))

------Argument Classification-------

weighted F1-score: 0.9034748904825386
micro F1-score: 0.9061371841155235
macro F1-score: 0.543282757668341


In [5]:
# Import data
df = pd.read_csv('/content/drive/MyDrive/app_reviews_all_annotated2.csv')
df = df[['review', 'argument_cat', 'decision_cat']]

# Remove missing rows
df = df.dropna()

df = df.groupby('argument_cat').filter(lambda x : len(x) > 1)
df = df.groupby('decision_cat').filter(lambda x : len(x) > 1)

# Convert to numeric for bert
df['Argument'] = pd.Categorical(df['argument_cat'])
df['Decision'] = pd.Categorical(df['decision_cat'])
df['argument_cat'] = df['Argument'].cat.codes
df['decision_cat'] = df['Decision'].cat.codes
#df = df[['review', 'argument_cat', 'decision_cat']]
df = df[['review', 'decision_cat']]
df

,review,decision_cat
5,Everyone should use it Full Review,0
37,Improve face detection please !!! Full Rev...,4
192,I think we just need more updates Full Rev...,4
201,if you're looking for an alternative for Pho...,3
221,it's the best photo editor app i've ever see...,4
...,...,...
45986,OMG IT'S SO AWESOME YOU SHOULD TRY IT Full...,3
46051,I litterly love this game i use to play this...,0
46076,"Please make more levels. Other wise, it's go...",4
46077,awesome game worth the money! Full Review,1


In [6]:
train_df, eval_df = train_test_split(df, test_size = 0.1, stratify=df[['decision_cat']])

In [7]:
# Create a ClassificationModel
model = ClassificationModel(
    "distilbert",
    "distilbert-base-uncased",
    num_labels=5,
    args={"reprocess_input_data": True, "overwrite_output_dir": True},
)

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier

  0%|          | 0/2488 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/311 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1384: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/277 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/35 [00:00<?, ?it/s]

In [8]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='weighted')

def f1_multiclass2(labels, preds):
    return f1_score(labels, preds, average='micro')

def f1_multiclass3(labels, preds):
    return f1_score(labels, preds, average='macro')
    
result, model_outputs, wrong_predictions = model.eval_model(eval_df, f1=f1_multiclass, acc=accuracy_score)
result2, model_outputs, wrong_predictions = model.eval_model(eval_df, f1=f1_multiclass2, acc=accuracy_score)
result3, model_outputs, wrong_predictions = model.eval_model(eval_df, f1=f1_multiclass3, acc=accuracy_score)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1384: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/277 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/277 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/277 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/35 [00:00<?, ?it/s]

In [9]:
print('------Decision Classification-------\n')
print('weighted F1-score: ' + str(result['f1']))
print('micro F1-score: ' + str(result2['f1']))
print('macro F1-score: ' + str(result3['f1']))

------Decision Classification-------

weighted F1-score: 0.8557294305681872
micro F1-score: 0.8592057761732852
macro F1-score: 0.812349979419524
